In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import tensorflow as tf
from tensorflow import keras
from dataset import Dataset

gpus = tf.config.list_physical_devices(device_type='GPU')
if gpus:
    tf.config.experimental.set_memory_growth(device=gpus[0], enable=True)

In [2]:
batch_size = 2
input_path = 'train_data/sensor/2018-11-29.csv'
target_dir = 'train_data/heatMat/'
label_maps = pd.read_csv('data_name_idx.txt', header=None, sep="\t")
trian_gen = Dataset(batch_size, input_path, target_dir, label_maps,trian=True)
val_gen = Dataset(batch_size, input_path, target_dir, label_maps,val=True)

In [ ]:
def build_model():
    inputs = keras.Input(shape=(8,23,27,1))

    x = keras.layers.Conv3D(64,(3,3,3),padding='same',activation='relu')(inputs)
    x = keras.layers.BatchNormalization()(x)

    x = keras.layers.Conv3D(64,(3,3,3),padding='same',activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    # x = keras.layers.Conv3D(64,(3,3,3),padding='same',activation='relu')(x)
    # x = keras.layers.BatchNormalization()(x)  
    x = keras.layers.Conv3D(64,(3,3,3),padding='same',activation='relu')(x)
    x_c1 = keras.layers.BatchNormalization()(x)

    x_c2 = keras.layers.Conv3DTranspose(64,(1,3,3),strides=(1, 2,2), padding='same')(x_c1)
    x_c2 = keras.layers.BatchNormalization()(x_c2)
    x_c2 = keras.layers.UpSampling3D(size=(1,2,2))(x_c1) + x_c2

    x_c2 = keras.layers.Conv3D(64,(1,3,3),padding='same',activation='relu')(x_c2)
    x_c2 = keras.layers.BatchNormalization()(x_c2)

    x_c3 = keras.layers.Conv3DTranspose(64,(1,3,3),strides=(1, 2,2), padding='same')(x_c2)
    x_c3 = keras.layers.BatchNormalization()(x_c3)
    x_c3 = keras.layers.UpSampling3D(size=(1,2,2))(x_c2) + x_c3

    x_c3 = keras.layers.Conv3D(64,(1,3,3),padding='same',activation='relu')(x_c3)
    x_c3 = keras.layers.BatchNormalization()(x_c3)

    x_c4 = keras.layers.Conv3DTranspose(64,(1,3,3),strides=(1, 2,2), padding='same')(x_c3)
    x_c4 = keras.layers.BatchNormalization()(x_c4)
    x_c4 = keras.layers.UpSampling3D(size=(1,2,2))(x_c3) + x_c4

    x_c4 = keras.layers.Conv3D(128,(1,3,3),padding='same',activation='relu')(x_c4)

    outputs = keras.layers.Conv3D(1,(1,3,3), padding='same')(x_c4)
    return keras.Model(inputs,outputs)

model = build_model()
model.summary()

In [ ]:
def my_loss(y_true, y_pred):
    y_true = tf.where(y_true<0.9,0.,1.)
    cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true,logits=y_pred)
    return tf.reduce_mean(cross_ent)

def my_loss_2(y_true, y_pred):
    y_pred_sig = tf.clip_by_value(tf.sigmoid(y_pred),1e-15,1. -1e-15)
    y_pos_indx = tf.where(tf.greater_equal(y_true, 0.8))
    y_pos_loss = - tf.pow(1. - y_pred_sig, 2) * tf.math.log(y_pred_sig)
    pos_loss = tf.reduce_sum(tf.gather_nd(y_pos_loss,y_pos_indx))

    y_neg_indx = tf.where(tf.less(y_true, 0.8))
    y_neg_loss = - tf.pow(1.- y_true, 4) * tf.pow(y_pred_sig, 2) * tf.math.log(1.-y_pred_sig)  / (1. - y_pred_sig)
    neg_loss = tf.reduce_sum(tf.gather_nd(y_neg_loss,y_neg_indx)) 
    return pos_loss + neg_loss

In [ ]:
initial_learning_rate = 1e-4
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=200, decay_rate=0.96, staircase=True
)
model.compile(
    loss=my_loss_2,
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
)

checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "model_5.h5", save_best_only=True
)

In [ ]:
epochs = 25
history = model.fit(
    trian_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=[checkpoint_cb,tensorbord_callback],
    shuffle=True
)

pd.DataFrame(history.history).plot()
plt.savefig('loss.png')

In [ ]:
x, y_true = val_gen[8]
# model.load_weights('model_5.h5')
y_pred = model(x)

y_pred_sig = tf.clip_by_value(tf.sigmoid(y_pred),1e-15,1. -1e-15)
y_pos_indx = tf.where(tf.greater_equal(y_true, 0.8))
y_pos_loss1 = - tf.pow(1. - y_pred_sig, 2) * tf.math.log(y_pred_sig)
y_pos_loss = tf.reduce_sum(tf.gather_nd(y_pos_loss1,y_pos_indx))
print(y_pos_loss.numpy())

y_neg_indx = tf.where(tf.less(y_true, 0.8))
y_neg_loss1 = - tf.pow(1.- y_true, 4) * tf.pow(y_pred_sig, 2) * tf.math.log(1.-y_pred_sig)  / (1. - y_pred_sig)
y_neg_loss = tf.reduce_sum(tf.gather_nd(y_neg_loss1,y_neg_indx))
print(y_neg_loss.numpy())